In [1]:
%%file planar_waveguide.py
import numpy as np
import multiprocessing as mp
import pprint

#Starting parameters
a =  3e-6 #Thickness of the core in m
x = np.linspace(0,a,100)  #Distance between the centre of the core to the interface with the cladding -a<=x<=a
n_0 = 1.455  #Refractive index of the cladding
n_1 = 1.47  #Refractive index of the core
n_s = 1.46  #Refractive index of the substrate (higher than n_0?)
mu_0 = (4*np.pi)*1e-7  #Permeability of free space
epsilon_0 = 8.854e-12  #Permittivity of free space
Lambda = 1.55e-6 #Wavelength of the light in m
phi = 0 #defining the variable


#Relative refractive index
delta = ((n_1**2) - (n_0**2)) / (2*n_1**2)

#Numerical aperture
NA = n_1 * np.sqrt(2*delta) #Also equal to theta max, typical waveguides NA=0.21

#Phi max
phi_max = NA / n_1

#Wavenumber or circular wavenumber
k = 2*np.pi / Lambda

#Eigenvalue equations preliminary
gamma = ((n_s**2) - (n_0**2)) / ((n_1**2) - (n_s**2))   #Asymmetry factor

v = np.sqrt((k**2) * (a**2) * (n_1**2 - n_s**2))  #Normalized frequency

w_ang = 2.998e8*k #Angular frequency

modes = {}  #Contains the modes and normalized wavenumbers u, w, w_prime, normalized propagation constant, etc

def work_splitter():
    if (v**2)/2<2:
        return ((0, 0, 0.25), (0, 0.25, 0.5), (0, 0.5, 0.75), (0, 0.75, 1))
    elif 2<(v**2)/2<3:
        return ((0, 0, 0.5), (0, 0.5, 1), (1, 0, 0.5), (1, 0.5, 1))
    elif 3<(v**2)/2<4:
        return ((0, 0, 0.5), (0, 0.5, 1), (1, 0, 1), (2, 0, 1))
    elif 3<(v**2)/2<5:
        return ((0, 0, 1), (1, 0, 1), (2, 0, 1), (3, 0, 1))

def mode_finder(m, start, end):
    step_size = 1e-6
    for b in np.arange(start,end,step_size):

        u = v*np.sqrt(1-b) #Normalized wavenumber
        w = v*np.sqrt(b)   #Normalized wavenumber
        w_prime = v*np.sqrt(b+gamma) #Normalized wavenumber
        #Page 30 of the book
        if np.absolute(2*v*np.sqrt(1-b) - (m*np.pi + np.arctan(np.sqrt(b/(1-b)))
         + np.arctan(np.sqrt((b+gamma) / (1-b))))) < 1e-4 and np.absolute(u**2+w**2-v**2) < 1e-4:
            phi = m*np.pi*0.5 + 0.5*np.arctan(w/u)-0.5*np.arctan(w_prime/u) #Angle in radians
            phi_deg = phi * 180 / np.pi  #Angle in degrees
            """if phi<0:
                phi_deg = (phi + np.pi/2)*180/np.pi
            elif 0<phi<np.pi/2:
                phi_deg = phi*180/np.pi
            else:
                phi_deg = 'Check angle?'  #Ask what happens if angle negative or over pi/2"""

            n_e = np.sqrt((n_s**2) + b*((n_1**2)-(n_s**2))) #Effective index
            beta = n_e * k
            kappa = np.sqrt((k**2)*(n_1**2)-(beta**2)) #Wavenumber along the x axis
            sigma = np.sqrt((beta**2)-(k**2)*(n_0**2)) #Wavenumber along the x axis
            xi = np.sqrt((beta**2)-(k**2)*(n_s**2)) #Wavenumber along the x axis
            modes[m]={'u':u, 'w':w, 'w_prime':w_prime, 'b':b, 'v':v, 'phi':phi,'phi_deg':phi_deg, 'n_e':n_e, 'beta':beta,
                     'kappa':kappa, 'sigma':sigma, 'xi':xi}
            break
        else:
            continue

    return modes


if __name__=="__main__":
    pool = mp.Pool(processes = mp.cpu_count()+1)
    results = pool.starmap(mode_finder,work_splitter())
    pool.close()
    pool.join()
    print(pprint.pformat(results))

Overwriting planar_waveguide.py


In [2]:
%%cmd
python planar_waveguide.py

Microsoft Windows [Version 10.0.10586]
(c) 2015 Microsoft Corporation. All rights reserved.

C:\Users\Eduardo\0-Folders\Python>python planar_waveguide.py
[{},
 {0: {'b': 0.73109100000664518,
      'beta': 5948018.3504254082,
      'kappa': 359819.46144770383,
      'n_e': 1.4673176092108329,
      'phi': -0.053785021347859785,
      'phi_deg': -3.0816547242534003,
      'sigma': 769086.32351394929,
      'u': 1.0794583843430963,
      'v': 2.0816285470305016,
      'w': 1.7798727494637672,
      'w_prime': 2.307258970541858,
      'xi': 593290.91648791847}},
 {1: {'b': 0.058192999999999995,
      'beta': 5920721.7489986029,
      'kappa': 673384.22373028018,
      'n_e': 1.4605838061884706,
      'phi': 1.3651241722399972,
      'phi_deg': 78.215853580641891,
      'sigma': 517220.94416592096,
      'u': 2.0201526711908424,
      'v': 2.0816285470305016,
      'w': 0.5021559447948637,
      'w_prime': 1.551662832497765,
      'xi': 167385.31493162061}},
 {}]

C:\Users\Eduardo\0-Folders

In [116]:
import numpy as np
np.exp(1)

2.7182818284590451